In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

file_name = "df_vector.pkl"

In [2]:
df_vector = pd.read_pickle(file_name)
df_vector.head()

,Theme,FeatureVector,WordVector
0,Mathematics,"[0.04838709677419355, 0.03225806451612903, 0.0...","[0, zero, number, numerical, digit, use, repre..."
Abiogenesis,Science,"[0.003003003003003003, 0.003003003003003003, 0...","[biology, abiogenesis, ‘, ’, greek, bios, ‘, l..."
Abortion,Society and social sciences,"[0.01282051282051282, 0.003205128205128205, 0....","[abortion, termination, pregnancy, removal, ex..."
Abraham Lincoln,People,"[0.0, 0.0030581039755351682, 0.006116207951070...","[abraham, lincoln, link, ən, february, 12, 180..."
Abraham,People,"[0.0, 0.005681818181818182, 0.0056818181818181...","[abraham, originally, abram, common, hebrew, p..."


In [3]:
categories = sorted(set(df_vector['Theme']))
i = 0
mapping = {}
for category in categories:
    mapping.update({category: i})
    i+=1

df_vector["T.Numerical"] = df_vector.Theme.map(mapping)
df_vector.head()

,Theme,FeatureVector,WordVector,T.Numerical
0,Mathematics,"[0.04838709677419355, 0.03225806451612903, 0.0...","[0, zero, number, numerical, digit, use, repre...",5
Abiogenesis,Science,"[0.003003003003003003, 0.003003003003003003, 0...","[biology, abiogenesis, ‘, ’, greek, bios, ‘, l...",8
Abortion,Society and social sciences,"[0.01282051282051282, 0.003205128205128205, 0....","[abortion, termination, pregnancy, removal, ex...",9
Abraham Lincoln,People,"[0.0, 0.0030581039755351682, 0.006116207951070...","[abraham, lincoln, link, ən, february, 12, 180...",6
Abraham,People,"[0.0, 0.005681818181818182, 0.0056818181818181...","[abraham, originally, abram, common, hebrew, p...",6


In [4]:
import pickle

with open('vocabulary.pkl', 'rb') as f:
    vocabulary = pickle.load(f)

vocabulary = np.array(vocabulary)

In [5]:
x_tmp = df_vector["FeatureVector"].to_numpy()
tmp = []
for x in x_tmp:
    tmp.append(x.reshape(1,-1))
tmp_feature_per_column = pd.DataFrame(np.concatenate(tmp), index=df_vector.index, columns=vocabulary)
tmp_feature_per_column

,use,include,also,world,century,one,form,many,state,large,...,determines,underwater,street,unfinished,shi,armenian,poetic,iliad,odyssey,homeric
0,0.048387,0.032258,0.016129,0.000000,0.000000,0.016129,0.000000,0.016129,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abiogenesis,0.003003,0.003003,0.000000,0.003003,0.000000,0.000000,0.006006,0.009009,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abortion,0.012821,0.003205,0.000000,0.016026,0.000000,0.003205,0.000000,0.000000,0.006410,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abraham Lincoln,0.000000,0.003058,0.006116,0.000000,0.000000,0.006116,0.003058,0.000000,0.027523,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abraham,0.000000,0.005682,0.005682,0.000000,0.011364,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Writing,0.002494,0.007481,0.004988,0.000000,0.002494,0.000000,0.014963,0.004988,0.002494,0.002494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yangtze,0.005051,0.005051,0.005051,0.025253,0.000000,0.015152,0.000000,0.000000,0.000000,0.010101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year,0.010101,0.000000,0.010101,0.000000,0.000000,0.005051,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zheng He,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_vector = pd.concat([df_vector, tmp_feature_per_column], axis=1)
df_vector

,Theme,FeatureVector,WordVector,T.Numerical,use,include,also,world,century,one,...,determines,underwater,street,unfinished,shi,armenian,poetic,iliad,odyssey,homeric
0,Mathematics,"[0.04838709677419355, 0.03225806451612903, 0.0...","[0, zero, number, numerical, digit, use, repre...",5,0.048387,0.032258,0.016129,0.000000,0.000000,0.016129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abiogenesis,Science,"[0.003003003003003003, 0.003003003003003003, 0...","[biology, abiogenesis, ‘, ’, greek, bios, ‘, l...",8,0.003003,0.003003,0.000000,0.003003,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abortion,Society and social sciences,"[0.01282051282051282, 0.003205128205128205, 0....","[abortion, termination, pregnancy, removal, ex...",9,0.012821,0.003205,0.000000,0.016026,0.000000,0.003205,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abraham Lincoln,People,"[0.0, 0.0030581039755351682, 0.006116207951070...","[abraham, lincoln, link, ən, february, 12, 180...",6,0.000000,0.003058,0.006116,0.000000,0.000000,0.006116,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abraham,People,"[0.0, 0.005681818181818182, 0.0056818181818181...","[abraham, originally, abram, common, hebrew, p...",6,0.000000,0.005682,0.005682,0.000000,0.011364,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Writing,Society and social sciences,"[0.0024937655860349127, 0.007481296758104738, ...","[write, medium, human, communication, involve,...",9,0.002494,0.007481,0.004988,0.000000,0.002494,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yangtze,Geography,"[0.005050505050505051, 0.005050505050505051, 0...","[yangtze, yangzi, english, simplify, chinese, ...",2,0.005051,0.005051,0.005051,0.025253,0.000000,0.015152,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year,Science,"[0.010101010101010102, 0.0, 0.0101010101010101...","[year, annus, orbital, period, planetary, body...",8,0.010101,0.000000,0.010101,0.000000,0.000000,0.005051,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zheng He,People,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[zheng, simplify, chinese, 郑和, traditional, ch...",6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Test Train Split For Vectors
No need to standerdize the data as the vectors are standardized

In [7]:
x_vector = df_vector.loc[:, "use":].copy()
y_vector = df_vector["T.Numerical"].copy()
x_vector.head()

,use,include,also,world,century,one,form,many,state,large,...,determines,underwater,street,unfinished,shi,armenian,poetic,iliad,odyssey,homeric
0,0.048387,0.032258,0.016129,0.000000,0.000000,0.016129,0.000000,0.016129,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abiogenesis,0.003003,0.003003,0.000000,0.003003,0.000000,0.000000,0.006006,0.009009,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abortion,0.012821,0.003205,0.000000,0.016026,0.000000,0.003205,0.000000,0.000000,0.006410,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abraham Lincoln,0.000000,0.003058,0.006116,0.000000,0.000000,0.006116,0.003058,0.000000,0.027523,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abraham,0.000000,0.005682,0.005682,0.000000,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
x_vec_train, x_vec_test, y_vec_train, y_vec_test = train_test_split(x_vector, y_vector, test_size=0.2, random_state=5)

Check for NaN

In [9]:
tmp = [x_vec_train, x_vec_test, y_vec_train, y_vec_test]

for e in tmp:
    print(e.isnull().values.any())

False
False
False
False


In [10]:
x_vec_test.head()

,use,include,also,world,century,one,form,many,state,large,...,determines,underwater,street,unfinished,shi,armenian,poetic,iliad,odyssey,homeric
Tea,0.000000,0.006944,0.006944,0.006944,0.020833,0.000000,0.00000,0.006944,0.0,0.006944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hippocrates,0.016807,0.000000,0.016807,0.000000,0.000000,0.008403,0.00000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Medication,0.024540,0.006135,0.006135,0.006135,0.000000,0.006135,0.00000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
International Red Cross and Red Crescent Movement,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Black Death,0.000000,0.000000,0.008621,0.008621,0.012931,0.008621,0.00431,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
y_vec_test.head()

Tea                                                  1
Hippocrates                                          6
Medication                                           3
International Red Cross and Red Crescent Movement    9
Black Death                                          4
Name: T.Numerical, dtype: int64

# Linear Regression

In [12]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

In [13]:
linear_regression = linear_model.LinearRegression()
linear_regression.fit(x_vec_train, y_vec_train)
coef = linear_regression.coef_
print("Coefficient:")
np.array([vocabulary, coef]).T

Coefficient:


array([['use', '2.3791646012036516'],
       ['include', '-9.88182827459756'],
       ['also', '-12.732727668777333'],
       ...,
       ['iliad', '0.7756652249437905'],
       ['odyssey', '-0.08843378303825482'],
       ['homeric', '4.93890605172706']], dtype='<U32')

# Q: What does it mean in this sense the coefficient?
As I understood: I.e. If I were to increase the `use` by +1 then the predicted value for use would increase by 2.38.

## Lab: _Run a Linear classifier on the training data set, mark the mean squared error (MSE). What hypothesis can you make about this data set being linearly separable or not?_

In [14]:
x_vec_train_pred = linear_regression.predict(x_vec_train)
model_error = mean_squared_error(y_vec_train, x_vec_train_pred)
print(f"MSE:\t{model_error:.50f}")

MSE:	0.00000000000000000000000000015125862104785390388707


## A: I guess yes?
This is model is for a multiple regression with 5000 features, I guess that makes the MSE low for the training prediction.


## Lab: _How well does the linear classifier generalize to new data?_

In [15]:
x_vec_test_pred = linear_regression.predict(x_vec_test)

In [16]:
model_error = mean_squared_error(y_vec_test, x_vec_test_pred)
print("MSE:\t", model_error)

MSE:	 5.735628599777146


### A: I don't know. It's hard to interpret as this is multiple regression with 5000 features. The linear regression is in a 5000D space, and it's hard to interpret an error in this environment. I have no idea whether this is good or bad.

# Logistic Regression

In [17]:
log_regression = LogisticRegression(solver="lbfgs", random_state=42)
log_regression.fit(x_vec_train, y_vec_train)
coef = log_regression.coef_
print("Coefficient:")
coef

Coefficient:


array([[ 0.00281593,  0.06788922,  0.03250149, ...,  0.00171438,
         0.00180322,  0.00173704],
       [ 0.11841402,  0.02530355,  0.01276888, ..., -0.00149299,
         0.00193733, -0.0014704 ],
       [-0.43057165, -0.09940201, -0.0773306 , ..., -0.00304977,
        -0.00285461, -0.00299882],
       ...,
       [-0.14287325,  0.01874336, -0.18116155, ..., -0.00597523,
        -0.00560071, -0.00587704],
       [ 0.13457328, -0.07854224,  0.17583684, ..., -0.00391911,
        -0.00367004, -0.00385882],
       [ 0.7551679 ,  0.09457228,  0.02353351, ..., -0.00264088,
        -0.00248359, -0.00259875]])

# Q: How to interpret coefficients for logistics regression?

_Use Logistic regression on your training set. Then measure the error on the training set._

In [18]:
x_vec_train_pred = log_regression.predict(x_vec_train)
model_error = mean_squared_error(y_vec_train, x_vec_train_pred)
print(f"MSE:\t{model_error:.50f}")

MSE:	13.33836477987421353930130862863734364509582519531250


## A: For Linear Regression the MSE for the training was:
0.00000000000000000000000000015125862104785390388707. Compared to that 13.33 is a lot. I assume for this reason Linear Regression was a stronger model?

## Lab: _How well does the logistic classifier generalize to new data?_

In [19]:
x_vec_test_pred = log_regression.predict(x_vec_test)
model_error = mean_squared_error(y_vec_test, x_vec_test_pred)
print("MSE:\t", model_error)

MSE:	 13.28643216080402


The MSE was lower for unseen data which is odd. I would have expect that the MSE for the unseen data would have been higher.

# Conclusion:
* For Linear regression the model didn't perform that well. The MSE for training was: 0.00000000000000000000000000015125862104785390388707 while for the unseen test data it was: 5.735628599777146. The difference is huge.
* For Logistic regression the model did perform well. The MSE was for training was: 13.338 while for the unseen data it was: 13.2864. The difference is low.
